In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 49.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=045c5063d59743a567c549d1ba44fc521c730e736e792fc2d6dc4aa3a205b381
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Jan_July = !ls "/content/drive/My Drive/Decadata/Jan_July"

In [ ]:
for i in Jan_July:
  print (i)

transaction_2020_01_01-05_transaction_2020_01_01-05.csv
transaction_2020_01_06-10_transaction_2020_01_06-10.csv
transaction_2020_01_11-15_transaction_2020_01_11-15.csv
transaction_2020_01_16-20_transaction_2020_01_16-20.csv
transaction_2020_01_21-25_transaction_2020_01_21-25.csv
transaction_2020_01_26-30_transaction_2020_01_26-30.csv
transaction_2020_02_01-04_transaction_2020_02_01-04.csv
transaction_2020_02_05-09_transaction_2020_02_05-09.csv
transaction_2020_02_10-14_transaction_2020_02_10-14.csv
transaction_2020_02_15-19_transaction_2020_02_15-19.csv
transaction_2020_02_20-24_transaction_2020_02_20-24.csv
transaction_2020_02_25-29_transaction_2020_02_25-29.csv
transaction_2020_03_01-05_transaction_2020_03_01-05.csv
transaction_2020_03_06-10_transaction_2020_03_06-10.csv
transaction_2020_03_11-13_transaction_2020_03_11-13.csv
transaction_2020_03_14-16_transaction_2020_03_14-16.csv
transaction_2020_03_17-19_transaction_2020_03_17-19.csv
transaction_2020_03_20-22_transaction_2020_03_20

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("csv_parquet").getOrCreate()

In [ ]:
import time
start = time.time()

for i in Jan_July:
  df = spark.read.option("delimiter", "|").csv("/content/drive/My Drive/Decadata/Jan_July/{}".format(i), header = True)
  new_columns = ["REG_NUM", "TXN", "DATE_KEY", "TXN_TIME", "STORE", "REWARDS", "TXN_LINE", "SCN_ID", "ITEM_CODE", "ITEM_UNIT_REGULAR_PRICE", "ITEM_UNIT_SALE_PRICE", "QTY", "ITEM_WEIGHT", "NUMBER_OF_UNITS_SCANNED_FOR_A_WEIGHTED_ITEM", "ITEM_DEAL_QUANTITY", "ITEM_SALE_QUANTITY", "ITEM_NET_SALES", "ITEM_GROSS_SALES", "ITEM_PACK_SIZE", "ITEM_COST"]
  df = df.toDF(*new_columns)
  df.write.parquet("/content/drive/My Drive/Decadata/Jan_July_Parquet/{}.parquet".format(i), mode='append')

end = time.time()
print ("Total time is" , end-start)


Total time is 9039.157768249512


In [ ]:
# import pandas as pd
# pd.read_parquet('/content/drive/My Drive/Decadata/Jan_July_Parquet/*.parquet', engine='pyarrow')

List_parquet = !ls "/content/drive/My Drive/Decadata/Jan_July_Parquet"

In [ ]:
List_parquet

['transaction_2020_01_01-05_transaction_2020_01_01-05.parquet',
 'transaction_2020_01_06-10_transaction_2020_01_06-10.parquet',
 'transaction_2020_01_11-15_transaction_2020_01_11-15.parquet',
 'transaction_2020_01_16-20_transaction_2020_01_16-20.parquet',
 'transaction_2020_01_21-25_transaction_2020_01_21-25.parquet',
 'transaction_2020_01_26-30_transaction_2020_01_26-30.parquet',
 'transaction_2020_02_01-04_transaction_2020_02_01-04.parquet',
 'transaction_2020_02_05-09_transaction_2020_02_05-09.parquet',
 'transaction_2020_02_10-14_transaction_2020_02_10-14.parquet',
 'transaction_2020_02_15-19_transaction_2020_02_15-19.parquet',
 'transaction_2020_02_20-24_transaction_2020_02_20-24.parquet',
 'transaction_2020_02_25-29_transaction_2020_02_25-29.parquet',
 'transaction_2020_03_01-05_transaction_2020_03_01-05.parquet',
 'transaction_2020_03_06-10_transaction_2020_03_06-10.parquet',
 'transaction_2020_03_11-13_transaction_2020_03_11-13.parquet',
 'transaction_2020_03_14-16_transaction_

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(spark)

In [ ]:
df = sqlContext.read.parquet("/content/drive/My Drive/Decadata/Jan_July_Parquet/transaction_2020_*.parquet")

In [ ]:
df

DataFrame[REG_NUM: string, TXN: string, DATE_KEY: string, TXN_TIME: string, STORE: string, REWARDS: string, TXN_LINE: string, SCN_ID: string, ITEM_CODE: string, ITEM_UNIT_REGULAR_PRICE: string, ITEM_UNIT_SALE_PRICE: string, QTY: string, ITEM_WEIGHT: string, NUMBER_OF_UNITS_SCANNED_FOR_A_WEIGHTED_ITEM: string, ITEM_DEAL_QUANTITY: string, ITEM_SALE_QUANTITY: string, ITEM_NET_SALES: string, ITEM_GROSS_SALES: string, ITEM_PACK_SIZE: string, ITEM_COST: string]

In [ ]:
row_number = df.count()


In [ ]:
row_number

1054121444

In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = 'bigqueryfec'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://transactionsbucket/


In [ ]:
bucket_name = "transactionsbucket"

!gsutil -m cp -r /content/drive/My\ Drive/Decadata/Jan_July_Parquet/* gs://transactionsbucket/


In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='bigqueryfec')

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "Transactions.txn_parquet"

job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)
uri = "gs://transactionsbucket/*.parquet"


In [ ]:
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1054121444 rows.
